In [1]:
#/usr/bin/python
# -*- coding: utf-8 -*-

import os
import sys
import json
import keys
import glob
import re
from PIL import Image
from io import BytesIO
from requests_oauthlib import OAuth1Session

In [2]:
# 画像を保存するディレクトリ
script_dir_path = os.path.dirname(os.path.abspath("__file__"))
image_dir_path = os.path.normpath(os.path.join(script_dir_path, "../images"))
path = os.path.exists(image_dir_path)
if not path:
    os.mkdir(image_dir_path)

since_id_path = script_dir_path + "/since_id.txt"

with open(since_id_path) as f:
    since_id = f.read()

In [3]:
#API認証情報
CK = keys.CK
CS = keys.CS
AT = keys.AT
ATS = keys.ATS
twitter = OAuth1Session(CK,CS,AT,ATS)

#API取得
search = "https://api.twitter.com/1.1/search/tweets.json?tweet_mode=extended"

# ツイートの取得設定
num = 10
keyword = '#VRChat' + ' -filter:retweets'
params = {'q' : keyword, 'count' : num, 'since_id' : since_id}
req = twitter.get(search, params = params)

In [4]:
def generate_image_num():
    image_num_max = 50
    image_path_list = sorted(glob.glob(image_dir_path + '/[0-9][0-9][0-9].jpg'))
    image_name_list = [os.path.basename(p) for p in sorted(image_path_list) if os.path.isfile(p)]

    if len(image_name_list) == 0:
        image_num = 0
    else:
        image_num_last = int(re.sub(r"\D", "", image_name_list[-1]))
        if image_num_last < image_num_max:
            image_num = image_num_last + 1
        else:
            # 一番古いファイルパス
            image_path_oldest = sorted(image_path_list, key=lambda f: os.stat(f).st_mtime, reverse=True)[-1]
            image_num = int(re.sub(r"\D", "", image_path_oldest))
    
    return image_num

In [5]:
if req.status_code == 200:
    tweets = json.loads(req.content)
    
    if tweets['statuses']:
        since_id = tweets['statuses'][0]['id']
        print(tweets['statuses'][0]['full_text'])
        with open(since_id_path, mode='w') as f:
            f.write(str(since_id))
    
    for tweet in tweets['statuses']:
        if 'extended_entities' in tweet:
            if 'media' in tweet['extended_entities']:
                for media in tweet['extended_entities']['media']:
                    # 画像を保存
                    media_urls = media['media_url']
                    downloads = twitter.get(media_urls).content
                    image = Image.open(BytesIO(downloads))
                    image_resize = image.resize((1024, 1024))
                    image_num = generate_image_num()
                    image_name = '{0:03d}'.format(image_num) + '.jpg'
                    image_path = image_dir_path + '/' + image_name
                    image_resize.save(image_path)

else:
    print("ERROR: %d" % req.status_code)